<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/modeling_and_prediction_Quantile_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM BASELINE MODEL

In diesem Skript wird ein erstes Modell erzeugt, um aus den gemssenen Globalstrahlungsdaten den UVI zu berechen.

In [1]:
# Verbinden mit der Google-Drive
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow
!pip install keras

In [3]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Funktionen:

In [4]:
# sMAPE Funktion zum brechnen des symmetric mean absolute percentage error
def smape(y_true, y_pred):
    diff = K.abs(y_true - y_pred)
    denom = K.abs(y_true) + K.abs(y_pred)
    return 200.0 * K.mean(diff / (denom + K.epsilon()))

## Import

In [5]:
# Pfad zur CSV-Datei auf Google-Drive
name_Messwerte = 'Messdaten_CAMS_GHI.csv'
name_Vorhersage = 'Vorhersagedaten_CAMS_VarIdx.csv'
folder_import = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'

In [6]:
# Pfad zum Abspeichern des trainierten Modells in Google-Drive

model_path = '/content/drive/My Drive/Colab_Notebooks/LSTM_Model/full_model.keras'
weights_path = '/content/drive/My Drive/Colab_Notebooks/LSTM_Model/model_weights.weights.h5'

# Pfad für den Testdatensatz

testdata_path_X = '/content/drive/My Drive/Colab_Notebooks/LSTM_Model/model_testdata_X.csv'
testdata_path_Y = '/content/drive/My Drive/Colab_Notebooks/LSTM_Model/model_testdata_Y.csv'

# Pfad für die Logdatei

text_file_path = "/content/drive/MyDrive/Colab_Notebooks/LSTM_Model/model_results.txt"

In [7]:
# Import der Messdaten
df_Messdaten = pd.read_csv(folder_import + name_Messwerte)

In [8]:
# Import der Vorhersagedaten
df_Vorhersage = pd.read_csv(folder_import + name_Vorhersage)

In [9]:
# Konvertiere die Zeitspalten in ein gemeinsames Format und führe einen Merge der Datensätze durch
df_Messdaten['Datetime'] = pd.to_datetime(df_Messdaten['Datetime'])
df_Vorhersage['Datetime'] = pd.to_datetime(df_Vorhersage['Datetime'])

In [10]:
# Spalten in den Messdaten
df_Messdaten.columns

Index(['Datetime', 'Observation_period', 'Clear_sky_GHI', 'Clear_sky_BHI',
       'GHI', 'BHI', 'UVI', 'UVA', 'UVB', 'erythem', 'Datum', 'Uhrzeit',
       'Messzeitpunkt', 'ghi', 'Dif', 'Glo_SPLite', 'Dir', 'Temp',
       'DiffGreater2', 'SZA', 'time_sin', 'time_cos', 'date_sin', 'date_cos',
       'Date', 'Hour'],
      dtype='object')

## Funktionsdefinition:

In [11]:
def quantile_loss(q, y_true, y_pred):
    """Berechnet den Pinball Loss für ein einzelnes Quantil q."""
    e = y_true - y_pred
    return tf.reduce_mean(tf.maximum(q * e, (q - 1) * e))

In [12]:
def multi_quantile_loss(y_true, y_pred):
    """Berechnet den Gesamt-Quantile-Loss für alle Quantile."""
    loss = 0
    for i, q in enumerate(quantiles):
        loss += quantile_loss(q, y_true, y_pred[:, i])  # Jedes Quantil einzeln berechnen
    return loss

## Config

In [13]:
model_name = "LSTM_mit_Vorhersagewerten_1Step"

In [14]:
# Beispiel: Nur 'SZA' als Feature und 'UVI' als Label
columns_X = ['Clear_sky_GHI', 'Clear_sky_BHI', 'GHI', 'BHI', 'Temp', 'SZA', 'time_sin', 'time_cos', 'date_sin', 'date_cos']
columns_y = ['UVI']

In [15]:
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]

In [16]:
model_config = {
    "units_1": 64,
    "units_2": 32,
    "dropout_rate": 0.1,
    "final_activation": "linear"
}

In [17]:
training_config = {
    "loss": multi_quantile_loss,
    "optimizer": "adam",
    "metrics": [
        "mse",
        "mae",
        "mape",
        keras.metrics.RootMeanSquaredError(name="rmse")
    ]
}

In [18]:
fit_config = {
    "epochs": 10,
    "batch_size": 32,
    "sequence_length": 16
}

## Setup

In [19]:
df = df_Messdaten.copy()  # deine Messdaten

total_length = len(df)
train_size   = round(total_length * 0.80)
val_size     = round(total_length * 0.10)
test_size    = total_length - train_size - val_size  # restliche 10%

# Skaliere X und Y separat (immer zuerst nur auf dem Trainingsbereich fitten!)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# 1) Nur Trainingsdatensatz extrahieren
train_df = df.iloc[:train_size]
# Fit des Scalers NUR auf Trainingsdaten
train_df[columns_X] = scaler_X.fit_transform(train_df[columns_X])
train_df[columns_y] = scaler_y.fit_transform(train_df[columns_y])

# 2) Für Validation
val_df = df.iloc[train_size : train_size + val_size].copy()
val_df[columns_X] = scaler_X.transform(val_df[columns_X])
val_df[columns_y] = scaler_y.transform(val_df[columns_y])

# 3) Für Test
test_df = df.iloc[train_size + val_size :].copy()
test_df[columns_X] = scaler_X.transform(test_df[columns_X])
test_df[columns_y] = scaler_y.transform(test_df[columns_y])

# Kontrolle
print("Train:", train_df.shape)
print("Val:", val_df.shape)
print("Test:", test_df.shape)

Train: (77386, 26)
Val: (9673, 26)
Test: (9673, 26)


<ipython-input-19-76f70d0c4b52>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[columns_X] = scaler_X.fit_transform(train_df[columns_X])
<ipython-input-19-76f70d0c4b52>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[columns_y] = scaler_y.fit_transform(train_df[columns_y])


In [20]:
def create_sequence(X, y, seq_length):
    sequences = []
    labels = []
    for stop_idx in range(seq_length, len(X)):
        x_seq = X.iloc[stop_idx-seq_length:stop_idx].values  # (seq_length, num_features)
        y_label = y.iloc[stop_idx]                           # Wert am Zeitpunkt 'stop_idx'
        sequences.append(x_seq)
        labels.append(y_label)
    return np.array(sequences), np.array(labels)

In [21]:
def create_sequence_OneStepAhead(X, y, seq_length):
    sequences = []
    labels = []
    # bis len(X)-1, damit "stop_idx + 1" nicht out-of-range ist
    for stop_idx in range(seq_length, len(X)-1):
        x_seq = X.iloc[stop_idx-seq_length:stop_idx].values
        y_label = y.iloc[stop_idx + 1]  # => label ist der nächste Zeitschritt
        sequences.append(x_seq)
        labels.append(y_label)
    return np.array(sequences), np.array(labels)

In [23]:
sequence_len = fit_config["sequence_length"]

X_train_seq, y_train_seq = create_sequence_OneStepAhead(train_df[columns_X], train_df[columns_y], sequence_len)
X_val_seq,   y_val_seq   = create_sequence_OneStepAhead(val_df[columns_X],   val_df[columns_y],   sequence_len)
X_test_seq,  y_test_seq  = create_sequence_OneStepAhead(test_df[columns_X],  test_df[columns_y],  sequence_len)

print("X_train_seq:", X_train_seq.shape, "y_train_seq:", y_train_seq.shape)
print("X_val_seq:", X_val_seq.shape,"y_val_seq:" y_val_seq.shape)
print("X_test_seq:", X_test_seq.shape,"y_test_seq:",  y_test_seq.shape)

(77369, 16, 10) (77369, 1)
(9656, 16, 10) (9656, 1)
(9656, 16, 10) (9656, 1)


In [24]:
X_test_2D = pd.DataFrame(X_test_seq.reshape(9656, -1))
y_test_2D = pd.DataFrame(y_test_seq.reshape(9656, -1))

In [25]:
# abspeichern der Testdaten in der Google-Drive
X_test_2D.to_csv(testdata_path_X, index = False)
y_test_2D.to_csv(testdata_path_Y, index = False)

In [26]:
input_shape = (16, 1)

In [27]:
def build_and_compile_model(model_config, training_config, input_shape):

    model = Sequential()

    # Erste LSTM-Schicht
    model.add(
        LSTM(
            units=model_config["units_1"],
            return_sequences=True,
            input_shape=input_shape
        )
    )
    model.add(Dropout(model_config["dropout_rate"]))

    # Zweite LSTM-Schicht
    model.add(LSTM(units=model_config["units_2"]))

    # Dense-Ausgangsschicht mit 3 Outputs (für 10%, 50%, 90% Quantil)
    model.add(Dense(len(quantiles), activation=model_config["final_activation"]))

    # Kompilieren mit custom loss
    model.compile(**training_config)

    return model

In [28]:
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])  # z.B. (16, 1)

model = build_and_compile_model(
    model_config=model_config,
    training_config=training_config,
    input_shape=input_shape
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 16, 64)              │          19,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 95,345 (372.45 KB)

 Trainable params: 31,781 (124.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 63,564 (248.30 KB)

In [29]:
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=fit_config["epochs"],
    batch_size=fit_config["batch_size"]
)

Epoch 1/10
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 29s 10ms/step - loss: 0.2364 - mae: 0.1794 - mape: 75398.4844 - mse: 0.0640 - rmse: 0.2529 - val_loss: 0.2751 - val_mae: 0.1950 - val_mape: 532.8042 - val_mse: 0.0653 - val_rmse: 0.2556
Epoch 2/10
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - loss: 0.2317 - mae: 0.1761 - mape: 59703.5547 - mse: 0.0623 - rmse: 0.2495 - val_loss: 0.2731 - val_mae: 0.1930 - val_mape: 549.3373 - val_mse: 0.0649 - val_rmse: 0.2547
Epoch 3/10
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 0.2311 - mae: 0.1759 - mape: 70074.9531 - mse: 0.0621 - rmse: 0.2493 - val_loss: 0.2708 - val_mae: 0.1959 - val_mape: 558.0349 - val_mse: 0.0659 - val_rmse: 0.2567
Epoch 4/10
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - loss: 0.2302 - mae: 0.1760 - mape: 75154.0625 - mse: 0.0620 - rmse: 0.2491 - val_loss: 0.2737 - val_mae: 0.1969 - val_mape: 547.1771 - val_mse: 0.0670 - val_rmse: 0.2589
Epoch 5/10
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.2302 - mae: 0.1755 - map

In [33]:
y_train_seq.shape

(77369, 1)

In [30]:
# 2) Evaluieren (Keras-integrierte Metriken)
val_results = model.evaluate(X_val_seq, y_val_seq, verbose=0)
test_results = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print("Model metrics names:", model.metrics_names)

# 3) Vorhersagen für weitere Kennzahlen (z.B. R2-Score)
y_val_pred = model.predict(X_val_seq).flatten()
y_test_pred = model.predict(X_test_seq).flatten()

val_r2  = r2_score(y_val_seq, y_val_pred)
test_r2 = r2_score(y_test_seq, y_test_pred)

# 4) Dictionary bauen
metrics = {
    "val_loss": val_results[0],
    "val_mse":  val_results[1],
    "val_mae":  val_results[2],
    "val_rmse": val_results[3],
    "test_loss": test_results[0],
    "test_mse":  test_results[1],
    "test_mae":  test_results[2],
    "test_rmse": test_results[3],
    "val_r2": val_r2,
    "test_r2": test_r2
}

Model metrics names: ['loss', 'compile_metrics']
302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


ValueError: Found input variables with inconsistent numbers of samples: [9656, 48280]

In [34]:
# Berechnen des vorhergesagten UVI
y_pred = model.predict(X_test_seq)

302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [35]:
y_pred = y_pred.flatten()
y_test_seq = y_test_seq.flatten()

In [36]:
type(y_pred)

numpy.ndarray

In [37]:
# Püfen, ob die Vorhersage den selben Shape hat wie
print(y_pred.shape, y_test_seq.shape)

(48280,) (9656,)


In [38]:
# Abspeichern des trainierten Modells und der Gewichte

model.save(model_path)
model.save_weights(weights_path)

# Abspeichern des Testdatensatzes für eine spätere Auswertung

## Berechnung der Qualitätsparameter:

In [39]:
def compare_model_output(model, X_test, y_test, plot_indices=None):

    # Vorhersage berechnen
    y_pred = model.predict(X_test)
    # Arrays abflachen für leichtere Handhabung
    y_pred = y_pred.flatten()
    y_test = y_test.flatten()

    # Metriken berechnen
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Ergebnis ausgeben
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")

    # Gesamten Vergleich plotten
    plt.figure(figsize=(12, 4))
    plt.plot(y_pred, label='Vorhersage', alpha=0.7)
    plt.plot(y_test, label='Tatsächlicher Wert', alpha=0.7)
    plt.title('Vergleich Vorhersage vs. Tatsächlicher Wert')
    plt.legend()
    plt.show()

    # Optionaler Detailplot für einen Index-Bereich
    if plot_indices is not None:
        start, end = plot_indices
        plt.figure(figsize=(12, 4))
        plt.plot(np.arange(start, end), y_pred[start:end], label='UVI Vorhersage', alpha=0.7)
        plt.plot(np.arange(start, end), y_test[start:end], label='UVI Messung', alpha=0.7)
        plt.title(f'Detailplot (Index {start} - {end})')
        plt.legend()
        plt.show()

In [40]:
compare_model_output(model, X_test_seq, y_test_seq, plot_indices=None)

302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


ValueError: Found input variables with inconsistent numbers of samples: [9656, 48280]

In [ ]:
compare_model_output(model, X_test_seq[:750], y_test_seq[:750], plot_indices=None)

In [ ]:
def log_model_results_to_csv(
    model_name,
    model_config,        # Dictionary mit Modell-Hyperparametern
    training_config,     # Dictionary mit Compile-Einstellungen (model.compile)
    fit_config,          # <--- NEU: Dictionary mit fit-Parametern (epochs, batch_size, sequence_length)
    metrics,             # Dictionary mit erzielten Metriken
    csv_path="model_results.csv"
):

    # 1) Zusammenführen aller Informationen in ein Dictionary
    results_dict = {
        "model_name": model_name
    }

    # Modell-Konfigurationswerte hinzufügen
    for k, v in model_config.items():
        results_dict[f"model_config.{k}"] = v

    # Trainings-Konfigurationswerte (compile) hinzufügen
    for k, v in training_config.items():
        results_dict[f"training_config.{k}"] = str(v)

    # Fit-Konfigurationswerte (fit) hinzufügen
    for k, v in fit_config.items():
        results_dict[f"fit_config.{k}"] = v

    # Metrics hinzufügen
    results_dict.update(metrics)

    # 2) Anzeige in der Konsole
    print("=== Model Results ===")
    for key, value in results_dict.items():
        print(f"{key}: {value}")
    print("======================\n")

    # 3) In einen DataFrame umwandeln (eine Zeile pro Modell)
    df_results = pd.DataFrame([results_dict])

    # 4) Schreiben oder Anhängen in die CSV
    if not os.path.exists(csv_path):
        df_results.to_csv(csv_path, index=False)
    else:
        df_results.to_csv(csv_path, mode='a', index=False, header=False)

In [ ]:
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])

In [ ]:
log_model_results_to_csv(
    model_name=model_name,
    model_config=model_config,
    training_config=training_config,
    fit_config=fit_config,        # <--- Neues Argument
    metrics=metrics,
    csv_path="model_results.csv"
)